In [1]:
deploy_phase = "local"
execution_date = '2024-07-06T00:00:00+09:00'
content_index = 0

# Notebook Initialization

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# import sys
# sys.path.insert(0, "/workspace/ojitong")

# Global Const

In [4]:
from datetime import datetime

def convert_to_datetime(date_str: str) -> datetime:
    return datetime.fromisoformat(date_str)

today_datetime = convert_to_datetime(execution_date)
today_datetime

datetime.datetime(2024, 7, 6, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))

In [5]:
import pandas as pd
from data.utils.file import generate_s3_path
s3_path = generate_s3_path(prefix="blog_contents", today_datetime=today_datetime)
df = pd.read_csv(s3_path)

In [6]:
content_row = df.iloc[content_index]

In [7]:
# 1. 필요한 데이터 추출
title = content_row['title']
publish_date = content_row['publish_date']
article = content_row['text']
category = content_row['category']

In [8]:
# 출력
print("Title:", title)
print("Publish Date:", publish_date)
print("Article:", article)
print("Category:", category)

Title: '시청역 역주행' 가해 차량, 보험이력 살펴보니…6년간 6번 사고
Publish Date: 2024-07-06 18:15:24+09:00
Article: [아이뉴스24 김효진 기자] 서울 중구 시청역 인근에서 9명을 숨지게 한 운전자 차모(68)씨가 몰던 제네시스 G80 차량이 사실상 매년 사고가 났던 것으로 알려졌다.

지난 1일 서울 시청역 인근 교차로에서 차량이 인도로 돌진해 9명이 숨졌다. 심하게 부서진 사고 차량. 2024.07.02. [사진=뉴시스]

5일 뉴시스 보도에 따르면 차씨가 몰던 G80 차량은 지난 2018년부터 2021년까지 최소 6회 교통사고로 차량이 파손돼 보험 처리를 한 것으로 나타났다.

이 사고들로 상대차 피해는 13회 발생했으며, 차씨 측이 부담한 상대차 수리(견적) 비용은 668만1847원에 달했다.

해당 차량은 2018년 9월에 첫 사고가 발생한 후, 이듬해인 2019년 2월 17일에 두 번째 사고가 났다. 또 2020년 10월에는 1일과 27일 각각 두 차례 사고가 났고, 1년 후인 2021년 11월과 12월에도 사고가 각각 발생했다.

차씨가 차량을 구입한 2018년부터 사실상 매해 사고가 발생한 셈이다.

차씨는 1974년 버스 면허를 취득했으며, 지난해 2월 3일자로 경기도 안산 소재 버스회사에 입사해 20인승 시내버스를 운전했다.

차씨가 근무했던 버스회사 측은 그에 대해 '무사고 경력자'라고 했으며, 차씨 아내 또한 사고 직후 남편에 대해 '베테랑 운전자'라고 말한 바 있다.

제네시스 G80 차량의 소유주는 가해자의 아내인 것으로 알려졌으나, 부부는 이 차량을 함께 사용한 것으로 전해졌다. 따라서 6번 사고의 운전자가 차씨와 아내 중 누구인지는 확인되지 않은 상태다.

앞서 지난 1일 오후 9시27분께 서울 지하철 2호선 시청역 인근 교차로에서 제네시스 G80 차량이 BMW, 소나타 등 차량을 차례로 친 후 횡단보도에 있던 보행자들을 덮치는 사고가 발생했다.

이 사고로 9명이 사망하고 7명이 다

In [9]:
publish_date = datetime.strptime(publish_date, '%Y-%m-%d %H:%M:%S%z')
formatted_date_str = publish_date.strftime('%Y-%m-%d %H:%M:%S')
source_content = title + formatted_date_str + article

In [ ]:
from data.evaluations.handler import EvaluationHandler
handler = EvaluationHandler(df, content_index, s3_path)

# Metric-based Evaluation

In [64]:
from data.const import PostType
from data.evaluations.metric_based import (
    evaluate_post_metric_based
)

In [67]:
recall_score = evaluate_post_metric_based(category, content_row["blog_content"])

사고 범주에 대한 평균 ROUGE Recall 점수: 1.0


In [ ]:
handler.eval_results["eval_metric_rouge1_score"] = recall_score

In [ ]:
handler.check_metric(recall_score, category)

# QGQA Evaluation

In [16]:
from data.tasks.evaluation_task import QGQAEvaluationTask
qgqa_task = QGQAEvaluationTask()

In [15]:
news_content = f"<뉴스 제목>: {title}, <뉴스 생성일자>: {publish_date}, <뉴스 본문>: {article}"

'<뉴스 제목>: \'시청역 역주행\' 가해 차량, 보험이력 살펴보니…6년간 6번 사고, <뉴스 생성일자>: 2024-07-06 18:15:24+09:00, <뉴스 본문>: [아이뉴스24 김효진 기자] 서울 중구 시청역 인근에서 9명을 숨지게 한 운전자 차모(68)씨가 몰던 제네시스 G80 차량이 사실상 매년 사고가 났던 것으로 알려졌다.\n\n지난 1일 서울 시청역 인근 교차로에서 차량이 인도로 돌진해 9명이 숨졌다. 심하게 부서진 사고 차량. 2024.07.02. [사진=뉴시스]\n\n5일 뉴시스 보도에 따르면 차씨가 몰던 G80 차량은 지난 2018년부터 2021년까지 최소 6회 교통사고로 차량이 파손돼 보험 처리를 한 것으로 나타났다.\n\n이 사고들로 상대차 피해는 13회 발생했으며, 차씨 측이 부담한 상대차 수리(견적) 비용은 668만1847원에 달했다.\n\n해당 차량은 2018년 9월에 첫 사고가 발생한 후, 이듬해인 2019년 2월 17일에 두 번째 사고가 났다. 또 2020년 10월에는 1일과 27일 각각 두 차례 사고가 났고, 1년 후인 2021년 11월과 12월에도 사고가 각각 발생했다.\n\n차씨가 차량을 구입한 2018년부터 사실상 매해 사고가 발생한 셈이다.\n\n차씨는 1974년 버스 면허를 취득했으며, 지난해 2월 3일자로 경기도 안산 소재 버스회사에 입사해 20인승 시내버스를 운전했다.\n\n차씨가 근무했던 버스회사 측은 그에 대해 \'무사고 경력자\'라고 했으며, 차씨 아내 또한 사고 직후 남편에 대해 \'베테랑 운전자\'라고 말한 바 있다.\n\n제네시스 G80 차량의 소유주는 가해자의 아내인 것으로 알려졌으나, 부부는 이 차량을 함께 사용한 것으로 전해졌다. 따라서 6번 사고의 운전자가 차씨와 아내 중 누구인지는 확인되지 않은 상태다.\n\n앞서 지난 1일 오후 9시27분께 서울 지하철 2호선 시청역 인근 교차로에서 제네시스 G80 차량이 BMW, 소나타 등 차량을 차례로 친 후 횡단보도에 있던 보행자들을 덮치는 사고가 발

In [17]:
content_row["blog_content"]

'### 제목: "서울 시청역 인근 대형 사고, 안전한 지하철 이용의 중요성"\n\n### 본문:\n안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 \'오.지.통 [오늘의 지하철 소식통]\' 인사 드립니다!\n\n**지연/사고 일시**\n2023년 10월 10일\n\n**지연/사고 노선**\n서울 시청역 인근\n\n**지연/사고 이유**\n도로 교통 사고 (Genesis G80 차량)\n\n**문의 사항 링크**\n[서울시 지하철 문의](https://www.seoulmetro.co.kr/)\n\n최근 서울 시청역 인근에서 발생한 대형 사고 소식이 전해져 안타까움을 자아내고 있습니다. 이번 사고는 Genesis G80 차량이 갑작스러운 가속으로 인해 발생한 것으로, 총 9명의 사망자와 7명의 부상자가 발생했어요. 차를 운전하던 68세의 차 모 씨는 브레이크가 갑자기 뻣뻣해져 제동이 어려웠다고 주장하고 있습니다. 경찰은 현재 이를 조사 중이며, 차 씨는 과거에도 최소 6회의 사고를 일으킨 전력이 있어 더욱 주목받고 있어요.\n\n차 씨는 오랜 경력의 버스 기사로 알려졌지만, 그의 사고 전력은 그와 상반되는 모습을 보여주고 있어요. 2018년에서 2021년 사이에 벌어진 6회의 사고로 총 13대의 다른 차량에 손해를 입힌 것으로 밝혀졌습니다. 이런 사고가 발생할 때마다 우리는 대중교통의 안전성에 대해 다시 한 번 생각하게 되네요.\n\n지하철은 상대적으로 안전한 대중교통 수단으로 평가받고 있으며, 사고나 지연이 발생할 확률이 낮습니다. 그러나 이번 사건은 우리에게 지하철 이용 시에도 안전에 대한 경각심을 가져야 함을 일깨워줍니다. 지하철을 이용할 때는 항상 안전수칙을 준수하고, 비상 상황에 대비해 비상구와 비상벨 위치를 미리 확인하는 것이 좋겠어요.\n\n마무리 말\n\n출퇴근 시간대에 지하철을 자주 이용하는 여러분께, 안전한 이용을 위한 정보를 제공하는 \'지하철 온다\' 블로그입니다. 이번 사고 소식을 전하면서 여러분의 안전을 다시 한번 강조하고 싶어요. 지

In [27]:
gqga_results = qgqa_task.execute(news_content, content_row["blog_content"])
gqga_results

{'date_comparison': False,
 'line_comparison': False,
 'news_details': '<뉴스> 발생일시: 2024년 7월 1일, 발생노선: 2호선',
 'blog_details': '<블로그> 발생일시: Unknown, 발생노선: Unknown'}

In [ ]:
eval_results.update({f"eval_qgqa_{key}": value for key, value in gqga_results.items()})

In [ ]:
handler.check_qgqa_eval(date_comparison, line_comparison)

# GEval Evaluation

In [61]:
from data.tasks.evaluation_task import GEvalEvaluationTask

In [62]:
geval_task = GEvalEvaluationTask()

In [63]:
geval_results = geval_task.execute(news_content, content_row["blog_content"])
geval_results

{'Consistency': {'score': 1,
  'reason': 'The blog content contains a different date (July 1, 2024) and mentions a different number of casualties (9 deaths and 7 injuries) compared to the article, which states the incident occurred on October 10, 2023. The blog content also incorrectly implies that the incident was a new occurrence in 2024, while the article discusses an event from 2023. The context of the blog does not logically follow from the information contained in the article.'},
 'Human_Likeness': {'score': 3,
  'reason': "The blog post provides detailed information about a tragic incident involving a vehicle crash, including statistics about the driver's accident history. While the content is informative and relevant, the tone is quite formal and lacks a conversational style. There are also some abrupt transitions between the paragraphs that disrupt the flow of the narrative. The writing could benefit from more engaging elements, such as personal anecdotes or rhetorical questio

In [ ]:
handler.eval_results.update(
    {
        f"eval_geval_{aspect.lower()}_{detail}": detail_value 
        for aspect, aspect_value in geval_results.items() 
        for detail, detail_value in aspect_value.items()
    }
)

In [ ]:
handler.check_geval_eval(geval_results)

# Evaluation Result 결과 업로드

In [ ]:
# 업데이트 된 파일 S3 업로드
s3_path = generate_s3_path(prefix="final_results", today_datetime=today_datetime)
df.to_csv(s3_path, index=False)
print(f"*** Data saved to {s3_path} ***")